## cl -cloud fraction (high clouds and low clouds)

In [99]:
import xarray as xr
import os
import numpy as np
import matplotlib.pyplot as plt

### get the variable

In [100]:
# folder = '/g/data/al33/replicas/CMIP5/combined/NOAA-GFDL/GFDL-CM3/historical/mon/atmos/Amon/r1i1p1/v20120227/cl'
# fileName = 'cl_Amon_GFDL-CM3_historical_r1i1p1_197001-197412.nc'
# path = folder + '/' + fileName
# ds = xr.open_dataset(path)
# ds

In [101]:
experiments=[
    # 'historical',
    'rcp85'
]
experiment = experiments[0]

path_gen = '/g/data/al33/replicas/CMIP5/combined/NOAA-GFDL/GFDL-CM3/' + experiment + '/mon/atmos/Amon'
ensemble = 'r1i1p1'
version = os.listdir(os.path.join(path_gen, ensemble))[-1]
path_folder =  os.path.join(path_gen, ensemble, version,'cl')

if experiment == 'historical':
    yearEnd_first = 1970
    yearStart_last = 1999

if experiment == 'rcp85':
    yearEnd_first = 2070
    yearStart_last = 2099


files = [f for f in os.listdir(path_folder) if f.endswith('.nc')]
files = sorted(files, key=lambda x: x[x.index(".nc")-13:x.index(".nc")-9])
files = [f for f in files if int(f[f.index(".nc")-13:f.index(".nc")-9]) <= yearStart_last and int(f[f.index(".nc")-6:f.index(".nc")-2]) >= yearEnd_first]

path_fileList = []
for file in files:
    path_fileList = np.append(path_fileList, os.path.join(path_folder, file))

ds = xr.open_mfdataset(path_fileList, combine='by_coords')

In [102]:
ds

<xarray.Dataset>
Dimensions:     (time: 420, lat: 90, bnds: 2, lon: 144, lev: 48)
Coordinates:
  * lat         (lat) float64 -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
  * lon         (lon) float64 1.25 3.75 6.25 8.75 ... 351.2 353.8 356.2 358.8
  * lev         (lev) float64 0.9961 0.9874 0.9767 ... 3.801e-05 1.711e-05
  * time        (time) object 2066-01-16 12:00:00 ... 2100-12-16 12:00:00
Dimensions without coordinates: bnds
Data variables: (12/14)
    average_DT  (time) timedelta64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    average_T1  (time) datetime64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    average_T2  (time) datetime64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    lat_bnds    (time, lat, bnds) float64 dask.array<chunksize=(60, 90, 2), meta=np.ndarray>
    lon_bnds    (time, lon, bnds) float64 dask.array<chunksize=(60, 144, 2), meta=np.ndarray>
    ps          (time, lat, lon) float32 dask.array<chunksize=(60, 90, 144), meta=np.ndarray>
    ...          ...
    lev_bnds    (time, lev, bnds) float64 dask.array<chunksize=(60, 48, 2), meta=np.ndarray>
    a           (time, lev) float64 dask.array<chunksize=(60, 48), meta=np.ndarray>
    a_bnds      (time, lev, bnds) float64 dask.array<chunksize=(60, 48, 2), meta=np.ndarray>
    b           (time, lev) float64 dask.array<chunksize=(60, 48), meta=np.ndarray>
    b_bnds      (time, lev, bnds) float64 dask.array<chunksize=(60, 48, 2), meta=np.ndarray>
    p0          (time) float32 1.013e+05 1.013e+05 ... 1.013e+05 1.013e+05
Attributes: (12/27)
    title:                  NOAA GFDL GFDL-CM3, RCP8.5 (run 1) experiment out...
    institute_id:           NOAA GFDL
    source:                 GFDL-CM3 2010 atmosphere: AM3 (AM3p9,C48L48); sea...
    contact:                gfdl.climate.model.info@noaa.gov
    project_id:             CMIP5
    table_id:               Table Amon (31 Jan 2011)
    ...                     ...
    parent_experiment_id:   historical
    physics_version:        1
    product:                output1
    institution:            NOAA GFDL(201 Forrestal Rd, Princeton, NJ, 08540)
    history:                File was processed by fremetar (GFDL analog of CM...
    parent_experiment_rip:  r1i1p1

### calculate indicies for pressure levels between 1000-600 hpa and 250-100 hpa

Pressure levels are given by:
<br />

$p(n,k,j,i) = a(k)p_0 + b(k)p_s(n,j,i)$
<br />

where \
n - time coordinate \
k - vertical coordinate \
i - horizontal coordinate 1 \
j - horizontal coordinate 2

In [103]:
pressureLevels = ds.a*ds.p0 + ds.b*ds.ps
pressureLevels

<xarray.DataArray (time: 420, lev: 48, lat: 90, lon: 144)>
dask.array<add, shape=(420, 48, 90, 144), dtype=float64, chunksize=(60, 48, 90, 144), chunktype=numpy.ndarray>
Coordinates:
  * lev      (lev) float64 0.9961 0.9874 0.9767 ... 3.801e-05 1.711e-05
  * time     (time) object 2066-01-16 12:00:00 ... 2100-12-16 12:00:00
  * lat      (lat) float64 -89.0 -87.0 -85.0 -83.0 -81.0 ... 83.0 85.0 87.0 89.0
  * lon      (lon) float64 1.25 3.75 6.25 8.75 11.25 ... 351.2 353.8 356.2 358.8

In [104]:
pressureLevels_low = xr.where((pressureLevels<=1000e2) & (pressureLevels>=600), 1, 0)
pressureLevels_high = xr.where((pressureLevels<=250e2) & (pressureLevels>=100), 1, 0)

### calculate low cloud and high cloud fraction

In [105]:
cl = ds.cl
cl

<xarray.DataArray 'cl' (time: 420, lev: 48, lat: 90, lon: 144)>
dask.array<concatenate, shape=(420, 48, 90, 144), dtype=float32, chunksize=(60, 48, 90, 144), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -89.0 -87.0 -85.0 -83.0 -81.0 ... 83.0 85.0 87.0 89.0
  * lon      (lon) float64 1.25 3.75 6.25 8.75 11.25 ... 351.2 353.8 356.2 358.8
  * lev      (lev) float64 0.9961 0.9874 0.9767 ... 3.801e-05 1.711e-05
  * time     (time) object 2066-01-16 12:00:00 ... 2100-12-16 12:00:00
Attributes:
    cell_methods:      time: mean
    interp_method:     conserve_order1
    long_name:         Cloud Area Fraction
    original_name:     tot_cloud_area
    units:             %
    standard_name:     cloud_area_fraction_in_atmosphere_layer
    original_units:    percent
    cell_measures:     area: areacella
    associated_files:  baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation...

In [106]:
cloud_low = cl*pressureLevels_low
cloud_low = cloud_low.max(dim='lev')

cloud_high = cl*pressureLevels_high
cloud_high = cloud_high.max(dim='lev')

In [107]:
cloud_high

<xarray.DataArray (time: 420, lat: 90, lon: 144)>
dask.array<_nanmax_skip-aggregate, shape=(420, 90, 144), dtype=float64, chunksize=(60, 90, 144), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -89.0 -87.0 -85.0 -83.0 -81.0 ... 83.0 85.0 87.0 89.0
  * lon      (lon) float64 1.25 3.75 6.25 8.75 11.25 ... 351.2 353.8 356.2 358.8
  * time     (time) object 2066-01-16 12:00:00 ... 2100-12-16 12:00:00

In [108]:
def regrid_conserv_np(M_in, M_out):

    # dimensions
    dlat = M_in.lat.data[1]-M_in.lat.data[0]
    dlon = M_in.lon.data[1]-M_in.lon.data[0]
    latBnds = (M_in.lat.data-(dlat/2), M_in.lat.data+(dlat/2))
    lonBnds = (M_in.lon.data-(dlon/2), M_in.lon.data+(dlon/2))
    lat = np.mean(latBnds, axis=0)
    lon = np.mean(lonBnds, axis=0)
    # area of gridboxes as fraction of earth surface area
    area_wlat = np.cos(np.deg2rad(lat))*dlat*np.pi/(4*180^2)

    dlat_n = M_out.lat.data[1]-M_out.lat.data[0]
    dlon_n = M_out.lon.data[1]-M_out.lon.data[0]
    latBnds_n = (M_out.lat.data-(dlat_n/2), M_out.lat.data+(dlat_n/2))
    lonBnds_n = (M_out.lon.data-(dlon_n/2), M_out.lon.data+(dlon_n/2))
    lat_n = np.mean(latBnds_n, axis=0)
    lon_n = np.mean(lonBnds_n, axis=0)

    # weights
    Wlat = np.zeros([len(lat_n), len(lat)])
    for i in np.arange(0,len(lat_n)):
        latBoxMin_n = latBnds_n[0][i]
        latBoxMax_n = latBnds_n[1][i]

        # gridboxes that are atleast partially overlapping with iteration gridbox
        J = (latBnds[0]<=latBoxMax_n)*(latBnds[1]>= latBoxMin_n)*area_wlat

        # including fractional area component contribution
        I = J*(latBnds[1]-latBoxMin_n)/dlat
        K = J*(latBoxMax_n-latBnds[0])/dlat
        II = np.min([I,J,K], axis=0)

        # weights from individual gridboxes contributing to the new gridbox as fraction of the total combined area contribution
        Wlat[i,:] = II/np.sum(II)

    Wlon = np.zeros([len(lon_n), len(lon)])
    for i in np.arange(0,len(lon_n)):
        lonBoxMin_n = lonBnds_n[0][i]
        lonBoxMax_n = lonBnds_n[1][i]

        # gridboxes that are atleast partially overlapping with iteration gridbox
        J = (lonBnds[0]<=lonBoxMax_n)*(lonBnds[1]>= lonBoxMin_n)*1

        # Including fractional area component contribution
        I = J*(lonBnds[1]-lonBoxMin_n)/dlon
        K = J*(lonBoxMax_n-lonBnds[0])/dlon
        L = J*(lonBoxMax_n-lonBnds[0]+360)/dlon
        II = np.min([I,J,K,L], axis=0)

        # weights from individual gridboxes contributing to the new gridbox as fraction of the total combined area contribution
        Wlon[i,:] = II/np.sum(II)

    # interpolation
    M_n = np.zeros([len(M_in.time.data), len(lat_n), len(lon_n)])
    for day in np.arange(0,len(M_in.time.data)):
        M_Wlat = np.zeros([len(lat_n), len(lon)])

        for i in range(0, np.shape(Wlat)[0]):
            M_Wlat[i,:] = np.nansum(M_in.isel(time=day) * np.vstack(Wlat[i,:]),axis=0)/np.sum(~np.isnan(M_in.isel(time=day))*1*np.vstack(Wlat[i,:]),axis=0)
            
        for i in range(0, np.shape(Wlon)[0]):
            M_n[day,:,i] = np.nansum(M_Wlat * Wlon[i,:],axis=1)/np.sum(~np.isnan(M_Wlat)*1*Wlon[i,:], axis=1)

    return M_n

In [82]:
M_out = xr.open_dataset('/g/data/k10/cb4968/data/cmip5/FGOALS-g2/FGOALS-g2_ds_regid_historical.nc')['pr']
M_out

<xarray.DataArray 'pr' (time: 1, lat: 22, lon: 128)>
[2816 values with dtype=float32]
Coordinates:
  * time     (time) object 1970-01-01 12:00:00
  * lat      (lat) float64 -29.3 -26.51 -23.72 -20.93 ... 20.93 23.72 26.51 29.3
  * lon      (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
Attributes:
    standard_name:     precipitation_flux
    long_name:         Precipitation
    comment:           at surface; includes both liquid and solid phases from...
    units:             kg m-2 s-1
    history:           Calculate as (1000* PRECT+1000*PRECSC+1000* PRECSL)
    cell_methods:      time: mean (interval: 20minutes)
    cell_measures:     area: areacella
    associated_files:  baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation...

In [ ]:
cloud_low_n = regrid_conserv_np(cloud_low, M_out)
cloud_low_n = xr.DataArray(
    data = cloud_low_n,
    dims = ['time', 'lat', 'lon'],
    coords = {'time': cloud_low.time.data, 'lat': M_out.lat.data, 'lon': M_out.lon.data},
    attrs = {'description': 'Maximum cloud fraction (%) from plev: 1000-600 hpa'}
    )
cloud_low_n

In [ ]:
cloud_high_n = regrid_conserv_np(cloud_high, M_out)
cloud_high_n = xr.DataArray(
    data = cloud_high_n,
    dims = ['time', 'lat', 'lon'],
    coords = {'time': cloud_high.time.data, 'lat': M_out.lat.data, 'lon': M_out.lon.data},
    attrs = {'description': 'Maximum cloud fraction (%) from plev: 250-100 hpa'}
    )

In [109]:
ds_clouds = xr.Dataset(
    data_vars = {
        'cloud_low': cloud_low, 
        'cloud_high':cloud_high},
    attrs = {'description': 'Metric defined as maximum cloud fraction (%) from specified pressure level intervals'}
    )

In [96]:
def save_file(dataset, folder, fileName):
    
    os.makedirs(folder, exist_ok=True)
    path = folder + '/' + fileName

    if os.path.exists(path):
        os.remove(path)    
    
    dataset.to_netcdf(path)

In [110]:
save = True
if save:
    folder = '/g/data/k10/cb4968/data/cmip5/ds'
    fileName = 'GFDL-CM3_cl_' + experiment + '.nc'

    save_file(ds_clouds, folder, fileName)